In [48]:
import requests
import pandas as pd

def make_fs_dataframe(firm_code):
    fs_url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode=' + firm_code
    fs_page = requests.get(fs_url)
    fs_tables = pd.read_html(fs_page.text)
    
    temp_df = fs_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df[temp_df.columns[:4]]
    temp_df = temp_df.loc[['매출액','영업이익','당기순이익']]
    
    temp_df2 = fs_tables[2]
    temp_df2 = temp_df2.set_index(temp_df2.columns[0])
    temp_df2 = temp_df2.loc[['자산','부채', '자본']]
    
    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]
    
    fs_df = pd.concat([temp_df, temp_df2, temp_df3])
    
    return fs_df

In [49]:
res = make_fs_dataframe('A005380')
print(res)

                2016/12    2017/12    2018/12    2019/12
IFRS(연결)                                                
매출액            936490.0   963761.0   968126.0  1057464.0
영업이익            51935.0    45747.0    24222.0    36055.0
당기순이익           57197.0    45464.0    16450.0    31856.0
자산            1788359.0  1781995.0  1806558.0  1945122.0
부채            1064914.0  1034421.0  1067597.0  1181465.0
자본             723446.0   747574.0   738960.0   763658.0
영업활동으로인한현금흐름     9970.0    39224.0    37643.0     4198.0


In [63]:
def change_df(firm_code, dataframe):
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code: dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]
        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index= True, right_index= True)
    
    return total_df

In [64]:
firmcode_list = ['A005930', 'A005380', 'A035420', 'A003550', 'A034730']

for num, code in enumerate(firmcode_list):
    fs_df = make_fs_dataframe(code)
    fs_df_changed = change_df(code, fs_df)
    if num == 0:
        total_fs = fs_df_changed
    else:
        total_fs = pd.concat([total_fs, fs_df_changed])

print(total_fs)

           2016/12                                                          \
IFRS(연결)     당기순이익        매출액         부채      영업이익 영업활동으로인한현금흐름         자본   
A005930   227261.0  2018667.0   692113.0  292407.0     473856.0  1929630.0   
A005380    57197.0   936490.0  1064914.0   51935.0       9970.0   723446.0   
A035420     7591.0    40226.0    22410.0   11020.0      11640.0    41296.0   
A003550    10913.0    96739.0    52694.0   13227.0       8782.0   142201.0   
A034730    28225.0   827299.0   607206.0   52808.0      91107.0   423267.0   

                      2017/12                        ...      2019/09  \
IFRS(연결)         자산     당기순이익        매출액         부채  ... 영업활동으로인한현금흐름   
A005930   2621743.0  421867.0  2395754.0   872607.0  ...          NaN   
A005380   1788359.0   45464.0   963761.0  1034421.0  ...          NaN   
A035420     63706.0    7701.0    46785.0    27141.0  ...       8184.0   
A003550    194894.0   24356.0   118411.0    54682.0  ...       7024.0   
A034730   10304